In [ ]:
# THIS PARAMETER IS USED FOR LOOPING OVER THE NOTEBOOK IN THE "run_all_customers" NOTEBOOK
system_id = 1

In [ ]:
# Base library imports
import pandas as pd
import torch
import os
from matplotlib import pyplot as plt

# SolNet imports
from src.data.datafetcher import PvFetcher
from src.data.featurisation import Featurisation
from src.tensors.tensorisation import Tensors
from src.models.lstm import LSTM
from src.models.training import Training
from src.models.training import save_model
from src.evaluation.evaluation import Evaluation

In [ ]:
# Hyperparameters needed for a run:

# Data fetching
locations_used = 1
start_date = 2005
end_date = 2010

# Forecasting parameters
target = 'P'
past_features = ['P']
future_features = ['hour_sin','hour_cos']
lags = 24
forecast_period = 24
gap = 0 
forecast_gap = 0

# Lstm parameters
hidden_size = 400
num_layers = 3
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training parameters
epochs = 200
batch_size = 32
learning_rate = 0.0001

## 1. Target location

In [ ]:
data_aus = pd.read_parquet('../data/australia/aus_production.parquet', engine='pyarrow')
data_aus = data_aus[data_aus['Customer'] == system_id]
data_aus

In [ ]:
# Hyperparams from the data
peak_power = data_aus['Generator Capacity'].iloc[0]
latitude = data_aus['latitude'].iloc[0]
longitude = data_aus['longitude'].iloc[0]

# Hyperparams not included in the data
tilt = 0
azimuth = 0
# The optimal angles replaces the tilt and azimuth by "ideal" settings
optimalangles = True

latitude, longitude, peak_power, tilt, azimuth

In [ ]:
# Unique name for the data, model and metrics
data_name = 'base_' + 'australia' '_' + str(system_id)
data_name

In [ ]:
# Create the folders to save the data and models
data_folder = '../results/AUS/'
model_folder = '../models/AUS/' + data_name
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
# Transform the dataframe to one retaining only the power output
data_aus = pd.DataFrame(data_aus['Values'])
data_aus = data_aus.resample('H').sum()
data_aus = data_aus.rename(columns={"Values":"P"})

target_data = data_aus
target_data

## 2. Source location

In [ ]:
# Fetch data from PVGIS
data_PVGIS = PvFetcher(latitude,longitude,peak_power, tilt, azimuth, locations=locations_used, start_date=start_date, end_date=end_date,optimal_angles=1)

In [ ]:
# Get the data from the fetcher
data = [data_PVGIS.dataset[0]]

# Localize the data so that hours align
data[0] = data[0].tz_localize('UTC').tz_convert('Australia/Sydney').tz_localize(None)

# Remove the hours before and after that do not make up a complete day
data[0] = data[0][13:-11]

# 3. Featurisation

## 3.1 Source

Cyclical features

In [ ]:
# Decide on the features to use in making the model (Note that 'P' should always be included since it's the target variable)
dataset = Featurisation(data).base_features(past_features)

# Include cyclical features
dataset = Featurisation(dataset).cyclic_features(yearly=False)
features = dataset[0].columns # update the features
source_data = dataset[0].copy()

## 3.2 Target

Cyclical features

In [ ]:
# Identical to the source domain
target_featurisation = Featurisation([target_data])
target_data = target_featurisation.cyclic_features()[0]

In [ ]:
# Include domain knowledge into the target domain for scaling purposes
# We know that the minimum power is always 0
domain_min = [0.0]
# We are going to assume that the maximum is the peak rated power times some degradation factor. In the paper we assume this degradation is 14%, this is the number also used by PVGIS
# cf. https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis/getting-started-pvgis/pvgis-user-manual_en#ref-9-hourly-solar-radiation-and-pv-data
domain_max = [peak_power*0.86]

# For other features we just assume that the minimum and maximum are what we have seen in the source data, this data is freely available, so this is not a stretch
other_features = past_features[1:] + future_features
for i in range(len(other_features)):
    domain_min.append(min(source_data[other_features[i]]))
    domain_max.append(max(source_data[other_features[i]]))

# 4. Tensors

## 4.1 Source

In [ ]:
# Get the data in the torch.tensor format
src_tensors = Tensors(source_data, 'P', past_features , future_features, lags, forecast_period, gap=gap, forecast_gap=forecast_gap)

# Split the data into train and test sets with separate tensors for features (X) and the target (y)
X_train_src, X_test_src, y_train_src, y_test_src = src_tensors.create_tensor()
X_train_src.shape, X_test_src.shape, y_train_src.shape, y_test_src.shape

## 4.2 Target

For the target dataset we require a separate "evaluation set" of a full year, apart from the train and test set. This makes the tensorisation of the data a bit more complex than what we did for the source domain.

In [ ]:
# Take apart the train and test data
target_excl_eval = target_data[:-365*24]

In [ ]:
# Get the months we have available for training. We need this info to make separate cases for each unique case of having "X months" of data in the target domain
training_months = list(target_excl_eval.index.month.unique())

In [ ]:
# the timestamps of the training start points for each case of having "X months" of data
train_starts = []
for i in range(len(training_months)):
    train_start = target_excl_eval[(target_excl_eval.index.month ==training_months[i])].index[0]
    train_starts.append(train_start)
    
train_starts = list(reversed(train_starts))

In [ ]:
# Get the target data in lists holding all the tensors for each of the "X months" cases. This time with a train and test set, as well as a separate evaluation set. 
X_train_target_list = []
X_test_target_list = []
X_eval_target_list = []
y_train_target_list = []
y_test_target_list = []
y_eval_target_list = []

for i in range(len(training_months)):
    tgt_tensors = Tensors(target_data.loc[train_starts[i]:], 'P', past_features , future_features, lags, forecast_period, gap=gap, forecast_gap=forecast_gap, evaluation_length=24*365, domain_min=domain_min, domain_max=domain_max)
    X_train_tgt, X_test_tgt, X_eval_tgt, y_train_tgt, y_test_tgt, y_eval_tgt = tgt_tensors.create_tensor()
    X_train_target_list.append(X_train_tgt)
    X_test_target_list.append(X_test_tgt)
    X_eval_target_list.append(X_eval_tgt)
    y_train_target_list.append(y_train_tgt)
    y_test_target_list.append(y_test_tgt)
    y_eval_target_list.append(y_eval_tgt) 
    print(X_train_tgt.shape, X_test_tgt.shape, X_eval_tgt.shape, y_train_tgt.shape, y_test_tgt.shape, y_eval_tgt.shape)

# 5. Source model

In [ ]:
# Set the parameters for the lstm
input_size = len(past_features + future_features)

my_lstm = LSTM(input_size,hidden_size,num_layers, forecast_period, dropout).to(device)
my_lstm

In [ ]:
# Initialize the trainer
training = Training(my_lstm, X_train_src, y_train_src, X_test_src, y_test_src, epochs,batch_size=batch_size, learning_rate=learning_rate)

# Train the model and return the trained parameters and the best iteration
state_dict_list, best_epoch = training.fit()

In [ ]:
# Load the state dictionary of the best performing model
my_lstm.load_state_dict(state_dict_list[best_epoch])

# Save the model state dictionary for later use 
save_model(my_lstm, 'AUS/' + data_name + '/model_' + data_name + '_transfer_0')

In [ ]:
# Forecast with the model
forecasts = my_lstm(X_test_src.to(device))

# Evaluate the model performance
source_eval = Evaluation(y_test_src.detach().flatten().numpy(), forecasts.cpu().detach().flatten().numpy())

# 6. Target model

In [ ]:
# Set the parameters for the lstm
input_size = len(past_features + future_features)

# Create empty models for each of the periods
target_lstm_list = []

for i in range(len(training_months)+1):
    target_lstm_list.append(LSTM(input_size,hidden_size,num_layers, forecast_period, dropout).to(device))

# The "0 months" case is basically random initialization of the weights, so we can already save this model as the target_0 model    
torch.save(target_lstm_list[0].state_dict(), '../models/AUS/' + data_name + '/model_' + data_name + '_target_0')

In [ ]:
# Keep track of the best performing iteration
target_best_epochs = [0]

for i in range(len(training_months)):
    # Initialize the trainer
    training = Training(target_lstm_list[i+1], X_train_target_list[i], y_train_target_list[i], X_test_target_list[i], y_test_target_list[i], epochs, learning_rate=learning_rate)

    # Train the model and return the trained parameters and the best iteration
    state_dict_list, best_epoch = training.fit()
    
    # Load the state dictionary of the best performing model
    target_lstm_list[i+1].load_state_dict(state_dict_list[best_epoch])
    target_best_epochs.append(best_epoch)

In [ ]:
# Maintain lists with all three evaluation metrics used in the paper
target_RMSEs = []
target_MBEs = []
target_MAEs = []

# Evaluate a clean model
forecasts = target_lstm_list[0](X_eval_target_list[0].to(device))
source_eval = Evaluation(y_eval_target_list[0].detach().flatten().numpy(), forecasts.cpu().detach().flatten().numpy())

target_RMSEs.append(source_eval.metrics()['RMSE'].values[0])
target_MBEs.append(source_eval.metrics()['MBE'].values[0])
target_MAEs.append(source_eval.metrics()['MAE'].values[0])

for i in range(len(training_months)):
    # Forecast with the model
    forecasts = target_lstm_list[i+1](X_eval_target_list[i].to(device))
    # Evaluate the model performance
    source_eval = Evaluation(y_eval_target_list[i].detach().flatten().numpy(), forecasts.cpu().detach().flatten().numpy())

    # Append the evaluation metrics
    target_RMSEs.append(source_eval.metrics()['RMSE'].values[0])
    target_MBEs.append(source_eval.metrics()['MBE'].values[0])
    target_MAEs.append(source_eval.metrics()['MAE'].values[0])

# 7. Transfer model

In [ ]:
# We freeze the weights and biases of the first layer
freezing = []

for name, _ in my_lstm.lstm.named_parameters():
    freezing.append(name)
    
freezing = freezing[:4]
freezing

In [ ]:
transfer_models = []
transfer_best_epochs = [0]

for i in range(len(training_months)):
    transfer_model  = LSTM(input_size,hidden_size,num_layers, forecast_period, dropout).to(device)
    transfer_model.load_state_dict(torch.load('../models/AUS/' + data_name + '/model_' + data_name + '_transfer_0'))
       
    for name, param in transfer_model.lstm.named_parameters():
        if any(freezing_name in name for freezing_name in freezing):
            param.requires_grad = False

    # Initialize the trainer
    training = Training(transfer_model, 
                              X_train_target_list[i], y_train_target_list[i], X_test_target_list[i], y_test_target_list[i], 
                              epochs=epochs, batch_size = batch_size, learning_rate =learning_rate/100)

    # Train the model and return the trained parameters and the best iteration
    state_dict_list, best_epoch = training.fit()
    
    # Load the state dictionary of the best performing model
    transfer_model.load_state_dict(state_dict_list[best_epoch])
    
    transfer_best_epochs.append(best_epoch)
    transfer_models.append(transfer_model)

In [ ]:
transfer_RMSEs = []
transfer_MBEs = []
transfer_MAEs = []

# Evaluate a clean model

transfer_model = LSTM(input_size,hidden_size,num_layers, forecast_period, dropout).to(device)
transfer_model.load_state_dict(torch.load('../models/AUS/' + data_name + '/model_' + data_name + '_transfer_0'))

forecasts = transfer_model(X_eval_target_list[0].to(device))
source_eval = Evaluation(y_eval_target_list[0].detach().flatten().numpy(), forecasts.cpu().detach().flatten().numpy())

transfer_RMSEs.append(source_eval.metrics()['RMSE'].values[0])
transfer_MBEs.append(source_eval.metrics()['MBE'].values[0])
transfer_MAEs.append(source_eval.metrics()['MAE'].values[0])

for i in range(len(training_months)):
    # Forecast with the model
    forecasts = transfer_models[i](X_eval_target_list[i].to(device))
    # Evaluate the model performance
    source_eval = Evaluation(y_eval_target_list[i].detach().flatten().numpy(), forecasts.cpu().detach().flatten().numpy())

    # Show the evaluation metrics
    transfer_RMSEs.append(source_eval.metrics()['RMSE'].values[0])
    transfer_MBEs.append(source_eval.metrics()['MBE'].values[0])
    transfer_MAEs.append(source_eval.metrics()['MAE'].values[0])

# 8. Baseline

In [ ]:
baseline_RMSEs = []
baseline_MBEs = []
baseline_MAEs = []

# Evaluate a clean model, our forecast in this case is basically the first feature in our features tensor, as we predict the next day to be the previous one 
forecasts = X_eval_target_list[0][:,:,0]
source_eval = Evaluation(y_eval_target_list[0].detach().flatten().numpy(), forecasts.cpu().detach().flatten().numpy())

baseline_RMSEs.append(source_eval.metrics()['RMSE'].values[0])
baseline_MBEs.append(source_eval.metrics()['MBE'].values[0])
baseline_MAEs.append(source_eval.metrics()['MAE'].values[0])

for i in range(len(training_months)):
    # Forecast with the model
    forecasts = X_eval_target_list[i][:,:,0]
    # Evaluate the model performance
    source_eval = Evaluation(y_eval_target_list[i].detach().flatten().numpy(), forecasts.cpu().detach().flatten().numpy())

    # Show the evaluation metrics
    baseline_RMSEs.append(source_eval.metrics()['RMSE'].values[0])
    baseline_MBEs.append(source_eval.metrics()['MBE'].values[0])
    baseline_MAEs.append(source_eval.metrics()['MAE'].values[0])

# 9. Final visualisation and export

In [ ]:
plt.plot(target_RMSEs,label='target')
plt.plot(transfer_RMSEs,label='transfer')
plt.plot(baseline_RMSEs, label='baseline')
plt.legend()

In [ ]:
column_names = []

for i in range(len(training_months)+1):
    column_names.append(str(i) + 'm')

In [ ]:
all_metrics = pd.DataFrame([baseline_RMSEs, target_RMSEs, transfer_RMSEs,
                            baseline_MBEs, target_MBEs, transfer_MBEs,
                            baseline_MAEs, target_MAEs, transfer_MAEs, 
                            target_best_epochs, transfer_best_epochs],
                           columns=column_names, index=['Baseline RMSE', 'Target RMSE', 'Transfer RMSE', 
                                                        'Baseline MBE', 'Target MBE', 'Transfer MBE', 
                                                        'Baseline MAE', 'Target MAE', 'Transfer MAE', 
                                                        'Target epoch', 'Transfer epoch']).transpose()

all_metrics['Target epoch'] = all_metrics['Target epoch'].astype(int)
all_metrics['Transfer epoch'] = all_metrics['Transfer epoch'].astype(int)
all_metrics

In [ ]:
all_metrics.to_csv('../results/AUS/' + 'summary_table_' + data_name + '.csv')

In [ ]:
data_name